In [51]:
import os
from time import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix
from collections import defaultdict
import pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [52]:
# Unpickle Function for the CIFAR-10 dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#save the model
def save_model(model):
    filename = 'SVM_model.sav'
    pickle.dump(model, open(filename, 'wb'))
    
def get_model():
    if os.path.isfile('SVM_model.sav'):
        svc=None
        flag_was_model_saved=False
        with open('SVM_model.sav', 'rb') as f:
            svc = pickle.load(f)
            flag_was_model_saved=True
        return [svc,flag_was_model_saved]
    
def combineData(data,limit=None):
    combined_data = defaultdict(list)
    for d in data:
        for key, value_list in d.items():
            if limit is not None:
                value_list = value_list[:limit]
            combined_data[key].extend(value_list)
    return combined_data

In [53]:
df=pd.read_csv('SVMs/LetterRec/letter-recognition.csv')
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [54]:
df.describe()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
count,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.023550,7.035500,5.121850,5.37245,3.505850,6.897600,7.500450,4.628600,5.178650,8.282050,6.45400,7.929000,3.046100,8.338850,3.691750,7.80120
std,1.913212,3.304555,2.014573,2.26139,2.190458,2.026035,2.325354,2.699968,2.380823,2.488475,2.63107,2.080619,2.332541,1.546722,2.567073,1.61747
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,3.000000,5.000000,4.000000,4.00000,2.000000,6.000000,6.000000,3.000000,4.000000,7.000000,5.00000,7.000000,1.000000,8.000000,2.000000,7.00000
50%,4.000000,7.000000,5.000000,6.00000,3.000000,7.000000,7.000000,4.000000,5.000000,8.000000,6.00000,8.000000,3.000000,8.000000,3.000000,8.00000
75%,5.000000,9.000000,6.000000,7.00000,5.000000,8.000000,9.000000,6.000000,7.000000,10.000000,8.00000,9.000000,4.000000,9.000000,5.000000,9.00000
max,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.00000


In [55]:
target=df['letter']
features=df.drop(['letter'],axis=1)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=42)

In [57]:
kernel=['linear','rbf','poly']
C=[0.1,1,10,100]
gamma=['auto',0.001,0.0001,0.1]
models=[]
index=0

pca=PCA(0.9)

X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)

for k in kernel:
    for c in C:
        for g in gamma:
            time_start=time()
            index+=1
            
            print('Kernel:',k,'C:',c,'gamma:',g,'index:',index)
            
            svc=SVC(kernel=k,C=c,gamma=g)
            svc.fit(X_train,y_train)
            f1=f1_score(y_test,svc.predict(X_test),average='macro')
            train_acccuracy=svc.score(X_train,y_train)
            test_accuracy=svc.score(X_test,y_test)
            
            time_end=time()
            print('Time Taken:',(time_end-time_start)/60,'minutes')
            models.append([svc,k,c,g,train_acccuracy,test_accuracy,f1,time_end-time_start])
    

Kernel: linear C: 0.1 gamma: auto index: 1
Time Taken: 0.2406500260035197 minutes
Kernel: linear C: 0.1 gamma: 0.001 index: 2
Time Taken: 0.24131666421890258 minutes
Kernel: linear C: 0.1 gamma: 0.0001 index: 3
Time Taken: 0.23919996817906697 minutes
Kernel: linear C: 0.1 gamma: 0.1 index: 4
Time Taken: 0.2376167376836141 minutes
Kernel: linear C: 1 gamma: auto index: 5
Time Taken: 0.2597666621208191 minutes
Kernel: linear C: 1 gamma: 0.001 index: 6
Time Taken: 0.26476656198501586 minutes
Kernel: linear C: 1 gamma: 0.0001 index: 7
Time Taken: 0.2663500706354777 minutes
Kernel: linear C: 1 gamma: 0.1 index: 8
Time Taken: 0.2621001044909159 minutes
Kernel: linear C: 10 gamma: auto index: 9
Time Taken: 0.5402167558670044 minutes
Kernel: linear C: 10 gamma: 0.001 index: 10
Time Taken: 0.5384666681289673 minutes
Kernel: linear C: 10 gamma: 0.0001 index: 11
Time Taken: 0.49128276109695435 minutes
Kernel: linear C: 10 gamma: 0.1 index: 12
Time Taken: 0.4868327339490255 minutes
Kernel: linear 

In [59]:
models.sort(key=lambda x: x[6],reverse=True)
from tabulate import tabulate
print(tabulate([x[1:] for x in models], headers=['Kernel', 'C','gamma','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
--------  -----  -------  ----------------  ---------------  ----------  ------------
rbf       100    0.1             1                 0.9325    0.932131        33.3404
rbf        10    0.1             0.999             0.932375  0.932015        34.195
rbf        10    auto            0.999417          0.93225   0.931888        32.01
rbf       100    auto            1                 0.932125  0.931743        35.9103
rbf         1    auto            0.979917          0.9285    0.928284        32.344
rbf         1    0.1             0.976667          0.927375  0.927086        32.225
poly        0.1  0.1             0.952333          0.88875   0.888755         8.93297
poly        0.1  auto            0.95775           0.887625  0.8876           8.42901
poly        1    0.1             0.986667          0.887625  0.88735          9.45307
poly        1    auto            0.990417          0.886625  0.88

In [21]:
"""       
20000 data
                                         Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf        10    auto            1                 0.971     0.970534          49.322
SVC(C=100, gamma='auto')                   rbf       100    auto            1                 0.971     0.970534          48.168
SVC(C=1, gamma='auto')                     rbf         1    auto            0.994083          0.965625  0.965049          50.95
SVC(C=10, gamma=0.1)                       rbf        10    0.1             1                 0.964375  0.963821          59.866
SVC(C=100, gamma=0.1)                      rbf       100    0.1             1                 0.964375  0.963821          60.613
SVC(C=1, gamma=0.1)                        rbf         1    0.1             0.997833          0.9635    0.962948          63.056
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999417          0.942875  0.942055          12.922
SVC(C=1, gamma=0.1, kernel='poly')         poly        1    0.1             1                 0.9415    0.940667          14.535
SVC(C=10, gamma=0.1, kernel='poly')        poly       10    0.1             1                 0.9415    0.940667          14.986
SVC(C=100, gamma=0.1, kernel='poly')       poly      100    0.1             1                 0.9415    0.940667          14.764
SVC(C=1, gamma='auto', kernel='poly')      poly        1    auto            1                 0.9415    0.940659          14.596
SVC(C=10, gamma='auto', kernel='poly')     poly       10    auto            1                 0.9415    0.940659          14.807
SVC(C=100, gamma='auto', kernel='poly')    poly      100    auto            1                 0.9415    0.940659          14.02
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             1                 0.9415    0.940658          14.89
SVC(C=100, gamma=0.001)                    rbf       100    0.001           0.951833          0.92625   0.925475          29.222
SVC(C=100, gamma=0.001, kernel='poly')     poly      100    0.001           0.945833          0.923     0.922226          17.44
SVC(C=10, gamma=0.001)                     rbf        10    0.001           0.893333          0.876875  0.875829          40.522
SVC(C=10, gamma=0.001, kernel='poly')      poly       10    0.001           0.886833          0.87225   0.87093           25.98
SVC(C=0.1, gamma='auto')                   rbf         0.1  auto            0.91175           0.869625  0.870333          66.596
SVC(C=0.1, gamma=0.1)                      rbf         0.1  0.1             0.908083          0.84      0.854825          71.873
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.879333          0.853625  0.851967          15.991
SVC(C=1, gamma=0.001, kernel='linear')     linear      1    0.001           0.879333          0.853625  0.851967          15.402
SVC(C=1, gamma=0.0001, kernel='linear')    linear      1    0.0001          0.879333          0.853625  0.851967          16.129
SVC(C=1, gamma=0.1, kernel='linear')       linear      1    0.1             0.879333          0.853625  0.851967          15.89
SVC(C=10, gamma='auto', kernel='linear')   linear     10    auto            0.88525           0.85275   0.850998          26.989
SVC(C=10, gamma=0.001, kernel='linear')    linear     10    0.001           0.88525           0.85275   0.850998          27.274
SVC(C=10, gamma=0.0001, kernel='linear')   linear     10    0.0001          0.88525           0.85275   0.850998          26.312
SVC(C=10, gamma=0.1, kernel='linear')      linear     10    0.1             0.88525           0.85275   0.850998          26.972
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.868417          0.849125  0.847425          19.661
SVC(C=0.1, gamma=0.001, kernel='linear')   linear      0.1  0.001           0.868417          0.849125  0.847425          19.947
SVC(C=0.1, gamma=0.0001, kernel='linear')  linear      0.1  0.0001          0.868417          0.849125  0.847425          17.098
SVC(C=0.1, gamma=0.1, kernel='linear')     linear      0.1  0.1             0.868417          0.849125  0.847425          17.216
SVC(C=100, gamma='auto', kernel='linear')  linear    100    auto            0.885417          0.848875  0.847202         128.015
SVC(C=100, gamma=0.001, kernel='linear')   linear    100    0.001           0.885417          0.848875  0.847202         125.273
SVC(C=100, gamma=0.0001, kernel='linear')  linear    100    0.0001          0.885417          0.848875  0.847202         124.849
SVC(C=100, gamma=0.1, kernel='linear')     linear    100    0.1             0.885417          0.848875  0.847202         126.325
SVC(C=100, gamma=0.0001)                   rbf       100    0.0001          0.860583          0.845875  0.844318          41.338
SVC(C=1, gamma=0.001)                      rbf         1    0.001           0.81125           0.802875  0.802706          62.707
SVC(C=10, gamma=0.0001)                    rbf        10    0.0001          0.798             0.7925    0.792276          59.253
SVC(C=1, gamma=0.001, kernel='poly')       poly        1    0.001           0.78975           0.785     0.783266          38.954
SVC(C=1, gamma=0.0001)                     rbf         1    0.0001          0.619             0.615875  0.599108          76.038
SVC(C=0.1, gamma=0.001)                    rbf         0.1  0.001           0.6135            0.61025   0.593612          74.197
SVC(C=0.1, gamma=0.001, kernel='poly')     poly        0.1  0.001           0.500583          0.499625  0.488087          40.966
SVC(C=100, gamma=0.0001, kernel='poly')    poly      100    0.0001          0.500583          0.499625  0.488087          50.365
SVC(C=0.1, gamma=0.0001)                   rbf         0.1  0.0001          0.0885833         0.08525   0.0210714         81.696
SVC(C=10, gamma=0.0001, kernel='poly')     poly       10    0.0001          0.0689167         0.064875  0.00937147        56.814
SVC(C=0.1, gamma=0.0001, kernel='poly')    poly        0.1  0.0001          0.0689167         0.064875  0.00937096        50.881
SVC(C=1, gamma=0.0001, kernel='poly')      poly        1    0.0001          0.0689167         0.064875  0.00937096        54.895

13000 data of 20000

                                           Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf        10    auto            1                0.93625    0.936904         9.58105
SVC(C=100, gamma='auto')                   rbf       100    auto            1                0.93625    0.936904         9.62596
SVC(C=10, gamma=0.1)                       rbf        10    0.1             1                0.930312   0.930981        10.746
SVC(C=100, gamma=0.1)                      rbf       100    0.1             1                0.930312   0.930981        10.6487
SVC(C=1, gamma='auto')                     rbf         1    auto            0.992917         0.92875    0.929662         9.83601
SVC(C=1, gamma=0.1)                        rbf         1    0.1             0.997917         0.925      0.925628        10.7666
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999792         0.896875   0.897533         2.47099
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             1                0.895312   0.896016         2.50957
SVC(C=1, gamma='auto', kernel='poly')      poly        1    auto            1                0.895312   0.896016         2.51843
SVC(C=1, gamma=0.1, kernel='poly')         poly        1    0.1             1                0.895312   0.896016         2.38981
SVC(C=10, gamma='auto', kernel='poly')     poly       10    auto            1                0.895312   0.896016         2.445
SVC(C=10, gamma=0.1, kernel='poly')        poly       10    0.1             1                0.895312   0.896016         2.49046
SVC(C=100, gamma='auto', kernel='poly')    poly      100    auto            1                0.895312   0.896016         2.50804
SVC(C=100, gamma=0.1, kernel='poly')       poly      100    0.1             1                0.895312   0.896016         2.41019
SVC(C=100, gamma=0.001, kernel='poly')     poly      100    0.001           0.949375         0.880625   0.881277         2.69
SVC(C=100, gamma=0.001)                    rbf       100    0.001           0.952917         0.88       0.880893         5.5551
SVC(C=10, gamma=0.001)                     rbf        10    0.001           0.884375         0.84375    0.844726         7.00016
SVC(C=10, gamma=0.001, kernel='poly')      poly       10    0.001           0.873125         0.834688   0.835082         3.4372
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.889583         0.82125    0.820917         3.64601
SVC(C=0.1, gamma=0.001, kernel='linear')   linear      0.1  0.001           0.889583         0.82125    0.820917         3.12898
SVC(C=0.1, gamma=0.0001, kernel='linear')  linear      0.1  0.0001          0.889583         0.82125    0.820917         3.666
SVC(C=0.1, gamma=0.1, kernel='linear')     linear      0.1  0.1             0.889583         0.82125    0.820917         3.503
SVC(C=100, gamma=0.0001)                   rbf       100    0.0001          0.855            0.818438   0.819266         7.46824
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.906875         0.817187   0.816731         3.20044
SVC(C=1, gamma=0.001, kernel='linear')     linear      1    0.001           0.906875         0.817187   0.816731         3.06699
SVC(C=1, gamma=0.0001, kernel='linear')    linear      1    0.0001          0.906875         0.817187   0.816731         3.162
SVC(C=1, gamma=0.1, kernel='linear')       linear      1    0.1             0.906875         0.817187   0.816731         3.26701
SVC(C=10, gamma='auto', kernel='linear')   linear     10    auto            0.921042         0.812813   0.81273          5.01901
SVC(C=10, gamma=0.001, kernel='linear')    linear     10    0.001           0.921042         0.812813   0.81273          5.71794
SVC(C=10, gamma=0.0001, kernel='linear')   linear     10    0.0001          0.921042         0.812813   0.81273          6.11601
SVC(C=10, gamma=0.1, kernel='linear')      linear     10    0.1             0.921042         0.812813   0.81273          4.9403
SVC(C=100, gamma='auto', kernel='linear')  linear    100    auto            0.927708         0.805937   0.806154        24.1335
SVC(C=100, gamma=0.001, kernel='linear')   linear    100    0.001           0.927708         0.805937   0.806154        20.6185
SVC(C=100, gamma=0.0001, kernel='linear')  linear    100    0.0001          0.927708         0.805937   0.806154        19.7791
SVC(C=100, gamma=0.1, kernel='linear')     linear    100    0.1             0.927708         0.805937   0.806154        19.8886
SVC(C=1, gamma=0.001)                      rbf         1    0.001           0.76375          0.755      0.76074         10.7371
SVC(C=0.1, gamma='auto')                   rbf         0.1  auto            0.827083         0.738125   0.74683         11.8595
SVC(C=10, gamma=0.0001)                    rbf        10    0.0001          0.754375         0.739062   0.744879         9.76772
SVC(C=1, gamma=0.001, kernel='poly')       poly        1    0.001           0.724375         0.704375   0.711509         4.80607
SVC(C=0.1, gamma=0.1)                      rbf         0.1  0.1             0.64625          0.529375   0.608442        12.3272
SVC(C=1, gamma=0.0001)                     rbf         1    0.0001          0.453125         0.430625   0.386974        12.2062
SVC(C=0.1, gamma=0.001)                    rbf         0.1  0.001           0.4225           0.4025     0.345189        12.078
SVC(C=100, gamma=0.0001, kernel='poly')    poly      100    0.0001          0.316667         0.309063   0.264459         5.80055
SVC(C=0.1, gamma=0.001, kernel='poly')     poly        0.1  0.001           0.31625          0.30875    0.264118         6.10557
SVC(C=0.1, gamma=0.0001)                   rbf         0.1  0.0001          0.0427083        0.0359375  0.00266852      12.374
SVC(C=0.1, gamma=0.0001, kernel='poly')    poly        0.1  0.0001          0.0427083        0.0359375  0.00266852       6.22876
SVC(C=1, gamma=0.0001, kernel='poly')      poly        1    0.0001          0.0427083        0.0359375  0.00266852       6.06409
SVC(C=10, gamma=0.0001, kernel='poly')     poly       10    0.0001          0.0427083        0.0359375  0.00266852       6.049
"       \n

8000 data of 20000\n\n\n                                           Kernel      C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken\n-----------------------------------------  --------  ---  -------  ----------------  ---------------  ----------  ------------\nSVC(C=10, gamma='auto')                    rbf        10  auto            1                0.93625    0.936904        10.6901\nSVC(C=100, gamma='auto')                   rbf       100  auto            1                0.93625    0.936904         9.76502\nSVC(C=1, gamma='auto')                     rbf         1  auto            0.992917         0.92875    0.929662        18.7002\nSVC(C=1, gamma='auto', kernel='poly')      poly        1  auto            1                0.895312   0.896016         2.54\nSVC(C=10, gamma='auto', kernel='poly')     poly       10  auto            1                0.895312   0.896016         2.53163\nSVC(C=100, gamma='auto', kernel='poly')    poly      100  auto            1                0.895312   0.896016         2.55097\nSVC(C=100, gamma=0.001, kernel='poly')     poly      100  0.001           0.949375         0.880625   0.881277         2.62706\nSVC(C=100, gamma=0.001)                    rbf       100  0.001           0.952917         0.88       0.880893         5.65757\nSVC(C=10, gamma=0.001)                     rbf        10  0.001           0.884375         0.84375    0.844726         8.409\nSVC(C=10, gamma=0.001, kernel='poly')      poly       10  0.001           0.873125         0.834688   0.835082         3.55242\nSVC(C=100, gamma=0.0001)                   rbf       100  0.0001          0.855            0.818438   0.819266         7.76601\nSVC(C=1, gamma='auto', kernel='linear')    linear      1  auto            0.906875         0.817187   0.816731         3.27099\nSVC(C=1, gamma=0.001, kernel='linear')     linear      1  0.001           0.906875         0.817187   0.816731         2.7393\nSVC(C=1, gamma=0.0001, kernel='linear')    linear      1  0.0001          0.906875         0.817187   0.816731         2.73722\nSVC(C=10, gamma='auto', kernel='linear')   linear     10  auto            0.921042         0.812813   0.81273          4.32653\nSVC(C=10, gamma=0.001, kernel='linear')    linear     10  0.001           0.921042         0.812813   0.81273          4.46051\nSVC(C=10, gamma=0.0001, kernel='linear')   linear     10  0.0001          0.921042         0.812813   0.81273          4.52612\nSVC(C=100, gamma='auto', kernel='linear')  linear    100  auto            0.927708         0.805937   0.806154        20.0262\nSVC(C=100, gamma=0.001, kernel='linear')   linear    100  0.001           0.927708         0.805937   0.806154        19.888\nSVC(C=100, gamma=0.0001, kernel='linear')  linear    100  0.0001          0.927708         0.805937   0.806154        23.5968\nSVC(C=1, gamma=0.001)                      rbf         1  0.001           0.76375          0.755      0.76074          9.944\nSVC(C=10, gamma=0.0001)                    rbf        10  0.0001          0.754375         0.739062   0.744879        10.035\nSVC(C=1, gamma=0.001, kernel='poly')       poly        1  0.001           0.724375         0.704375   0.711509         5.08002\nSVC(C=1, gamma=0.0001)                     rbf         1  0.0001          0.453125         0.430625   0.386974        11.994\nSVC(C=100, gamma=0.0001, kernel='poly')    poly      100  0.0001          0.316667         0.309063   0.264459         6.13449\nSVC(C=1, gamma=0.0001, kernel='poly')      poly        1  0.0001          0.0427083        0.0359375  0.00266852       6.43137\nSVC(C=10, gamma=0.0001, kernel='poly')     poly       10  0.0001          0.0427083        0.0359375  0.00266852       6.36198\n\n\n2000 of 20000\nKernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken\n-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------\nSVC(C=10, gamma='auto')                    rbf        10    auto            1                0.93625    0.936904        10.4032\nSVC(C=100, gamma='auto')                   rbf       100    auto            1                0.93625    0.936904        10.406\nSVC(C=10, gamma=0.1)                       rbf        10    0.1             1                0.930312   0.930981        11.6225\nSVC(C=100, gamma=0.1)                      rbf       100    0.1             1                0.930312   0.930981        11.8267\nSVC(C=1, gamma='auto')                     rbf         1    auto            0.992917         0.92875    0.929662        11.38\nSVC(C=1, gamma=0.1)                        rbf         1    0.1             0.997917         0.925      0.925628        11.5156\nSVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999792         0.896875   0.897533         2.70734\nSVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             1                0.895312   0.896016         2.674\nSVC(C=1, gamma='auto', kernel='poly')      poly        1    auto            1                0.895312   0.896016         2.67328\nSVC(C=1, gamma=0.1, kernel='poly')         poly        1    0.1             1                0.895312   0.896016         2.93201\nSVC(C=10, gamma='auto', kernel='poly')     poly       10    auto            1                0.895312   0.896016         2.87371\nSVC(C=10, gamma=0.1, kernel='poly')        poly       10    0.1             1                0.895312   0.896016         2.63401\nSVC(C=100, gamma='auto', kernel='poly')    poly      100    auto            1                0.895312   0.896016         2.71799\nSVC(C=100, gamma=0.1, kernel='poly')       poly      100    0.1             1                0.895312   0.896016         2.619\nSVC(C=100, gamma=0.001, kernel='poly')     poly      100    0.001           0.949375         0.880625   0.881277         2.993\nSVC(C=100, gamma=0.001)                    rbf       100    0.001           0.952917         0.88       0.880893         5.73056\nSVC(C=10, gamma=0.001)                     rbf        10    0.001           0.884375         0.84375    0.844726         7.70887\nSVC(C=10, gamma=0.001, kernel='poly')      poly       10    0.001           0.873125         0.834688   0.835082         4.528\nSVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.889583         0.82125    0.820917         3.53565\nSVC(C=0.1, gamma=0.001, kernel='linear')   linear      0.1  0.001           0.889583         0.82125    0.820917         3.81701\nSVC(C=0.1, gamma=0.0001, kernel='linear')  linear      0.1  0.0001          0.889583         0.82125    0.820917         3.36999\nSVC(C=0.1, gamma=0.1, kernel='linear')     linear      0.1  0.1             0.889583         0.82125    0.820917         3.942\nSVC(C=100, gamma=0.0001)                   rbf       100    0.0001          0.855            0.818438   0.819266         7.95704\nSVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.906875         0.817187   0.816731         3.08272\nSVC(C=1, gamma=0.001, kernel='linear')     linear      1    0.001           0.906875         0.817187   0.816731         3.73897\nSVC(C=1, gamma=0.0001, kernel='linear')    linear      1    0.0001          0.906875         0.817187   0.816731         3.13279\nSVC(C=1, gamma=0.1, kernel='linear')       linear      1    0.1             0.906875         0.817187   0.816731         3.82483\nSVC(C=10, gamma='auto', kernel='linear')   linear     10    auto            0.921042         0.812813   0.81273          5.843\nSVC(C=10, gamma=0.001, kernel='linear')    linear     10    0.001           0.921042         0.812813   0.81273          4.74299\nSVC(C=10, gamma=0.0001, kernel='linear')   linear     10    0.0001          0.921042         0.812813   0.81273          4.262\nSVC(C=10, gamma=0.1, kernel='linear')      linear     10    0.1             0.921042         0.812813   0.81273          5.27188\nSVC(C=100, gamma='auto', kernel='linear')  linear    100    auto            0.927708         0.805937   0.806154        20.766\nSVC(C=100, gamma=0.001, kernel='linear')   linear    100    0.001           0.927708         0.805937   0.806154        19.9108\nSVC(C=100, gamma=0.0001, kernel='linear')  linear    100    0.0001          0.927708         0.805937   0.806154        20.3163\nSVC(C=100, gamma=0.1, kernel='linear')     linear    100    0.1             0.927708         0.805937   0.806154        22.613\nSVC(C=1, gamma=0.001)                      rbf         1    0.001           0.76375          0.755      0.76074         10.6623\nSVC(C=0.1, gamma='auto')                   rbf         0.1  auto            0.827083         0.738125   0.74683         12.655\nSVC(C=10, gamma=0.0001)                    rbf        10    0.0001          0.754375         0.739062   0.744879        10.5508\nSVC(C=1, gamma=0.001, kernel='poly')       poly        1    0.001           0.724375         0.704375   0.711509         5.73216\nSVC(C=0.1, gamma=0.1)                      rbf         0.1  0.1             0.64625          0.529375   0.608442        12.719\nSVC(C=1, gamma=0.0001)                     rbf         1    0.0001          0.453125         0.430625   0.386974        12.712\nSVC(C=0.1, gamma=0.001)                    rbf         0.1  0.001           0.4225           0.4025     0.345189        12.522\nSVC(C=100, gamma=0.0001, kernel='poly')    poly      100    0.0001          0.316667         0.309063   0.264459         6.517\nSVC(C=0.1, gamma=0.001, kernel='poly')     poly        0.1  0.001           0.31625          0.30875    0.264118         6.17167\nSVC(C=0.1, gamma=0.0001)                   rbf         0.1  0.0001          0.0427083        0.0359375  0.00266852      12.838\nSVC(C=0.1, gamma=0.0001, kernel='poly')    poly        0.1  0.0001          0.0427083        0.0359375  0.00266852       6.71629\nSVC(C=1, gamma=0.0001, kernel='poly')      poly        1    0.0001          0.0427083        0.0359375  0.00266852       7.087\nSVC(C=10, gamma=0.0001, kernel='poly')     poly       10    0.0001          0.0427083        0.0359375  0.00266852       6.309\n"

8000 data of 20000


                                           Kernel      C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  ---  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf        10  auto            1                0.93625    0.936904        10.6901
SVC(C=100, gamma='auto')                   rbf       100  auto            1                0.93625    0.936904         9.76502
SVC(C=1, gamma='auto')                     rbf         1  auto            0.992917         0.92875    0.929662        18.7002
SVC(C=1, gamma='auto', kernel='poly')      poly        1  auto            1                0.895312   0.896016         2.54
SVC(C=10, gamma='auto', kernel='poly')     poly       10  auto            1                0.895312   0.896016         2.53163
SVC(C=100, gamma='auto', kernel='poly')    poly      100  auto            1                0.895312   0.896016         2.55097
SVC(C=100, gamma=0.001, kernel='poly')     poly      100  0.001           0.949375         0.880625   0.881277         2.62706
SVC(C=100, gamma=0.001)                    rbf       100  0.001           0.952917         0.88       0.880893         5.65757
SVC(C=10, gamma=0.001)                     rbf        10  0.001           0.884375         0.84375    0.844726         8.409
SVC(C=10, gamma=0.001, kernel='poly')      poly       10  0.001           0.873125         0.834688   0.835082         3.55242
SVC(C=100, gamma=0.0001)                   rbf       100  0.0001          0.855            0.818438   0.819266         7.76601
SVC(C=1, gamma='auto', kernel='linear')    linear      1  auto            0.906875         0.817187   0.816731         3.27099
SVC(C=1, gamma=0.001, kernel='linear')     linear      1  0.001           0.906875         0.817187   0.816731         2.7393
SVC(C=1, gamma=0.0001, kernel='linear')    linear      1  0.0001          0.906875         0.817187   0.816731         2.73722
SVC(C=10, gamma='auto', kernel='linear')   linear     10  auto            0.921042         0.812813   0.81273          4.32653
SVC(C=10, gamma=0.001, kernel='linear')    linear     10  0.001           0.921042         0.812813   0.81273          4.46051
SVC(C=10, gamma=0.0001, kernel='linear')   linear     10  0.0001          0.921042         0.812813   0.81273          4.52612
SVC(C=100, gamma='auto', kernel='linear')  linear    100  auto            0.927708         0.805937   0.806154        20.0262
SVC(C=100, gamma=0.001, kernel='linear')   linear    100  0.001           0.927708         0.805937   0.806154        19.888
SVC(C=100, gamma=0.0001, kernel='linear')  linear    100  0.0001          0.927708         0.805937   0.806154        23.5968
SVC(C=1, gamma=0.001)                      rbf         1  0.001           0.76375          0.755      0.76074          9.944
SVC(C=10, gamma=0.0001)                    rbf        10  0.0001          0.754375         0.739062   0.744879        10.035
SVC(C=1, gamma=0.001, kernel='poly')       poly        1  0.001           0.724375         0.704375   0.711509         5.08002
SVC(C=1, gamma=0.0001)                     rbf         1  0.0001          0.453125         0.430625   0.386974        11.994
SVC(C=100, gamma=0.0001, kernel='poly')    poly      100  0.0001          0.316667         0.309063   0.264459         6.13449
SVC(C=1, gamma=0.0001, kernel='poly')      poly        1  0.0001          0.0427083        0.0359375  0.00266852       6.43137
SVC(C=10, gamma=0.0001, kernel='poly')     poly       10  0.0001          0.0427083        0.0359375  0.00266852       6.36198


2000 of 20000
Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf        10    auto            1                0.93625    0.936904        10.4032
SVC(C=100, gamma='auto')                   rbf       100    auto            1                0.93625    0.936904        10.406
SVC(C=10, gamma=0.1)                       rbf        10    0.1             1                0.930312   0.930981        11.6225
SVC(C=100, gamma=0.1)                      rbf       100    0.1             1                0.930312   0.930981        11.8267
SVC(C=1, gamma='auto')                     rbf         1    auto            0.992917         0.92875    0.929662        11.38
SVC(C=1, gamma=0.1)                        rbf         1    0.1             0.997917         0.925      0.925628        11.5156
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999792         0.896875   0.897533         2.70734
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             1                0.895312   0.896016         2.674
SVC(C=1, gamma='auto', kernel='poly')      poly        1    auto            1                0.895312   0.896016         2.67328
SVC(C=1, gamma=0.1, kernel='poly')         poly        1    0.1             1                0.895312   0.896016         2.93201
SVC(C=10, gamma='auto', kernel='poly')     poly       10    auto            1                0.895312   0.896016         2.87371
SVC(C=10, gamma=0.1, kernel='poly')        poly       10    0.1             1                0.895312   0.896016         2.63401
SVC(C=100, gamma='auto', kernel='poly')    poly      100    auto            1                0.895312   0.896016         2.71799
SVC(C=100, gamma=0.1, kernel='poly')       poly      100    0.1             1                0.895312   0.896016         2.619
SVC(C=100, gamma=0.001, kernel='poly')     poly      100    0.001           0.949375         0.880625   0.881277         2.993
SVC(C=100, gamma=0.001)                    rbf       100    0.001           0.952917         0.88       0.880893         5.73056
SVC(C=10, gamma=0.001)                     rbf        10    0.001           0.884375         0.84375    0.844726         7.70887
SVC(C=10, gamma=0.001, kernel='poly')      poly       10    0.001           0.873125         0.834688   0.835082         4.528
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.889583         0.82125    0.820917         3.53565
SVC(C=0.1, gamma=0.001, kernel='linear')   linear      0.1  0.001           0.889583         0.82125    0.820917         3.81701
SVC(C=0.1, gamma=0.0001, kernel='linear')  linear      0.1  0.0001          0.889583         0.82125    0.820917         3.36999
SVC(C=0.1, gamma=0.1, kernel='linear')     linear      0.1  0.1             0.889583         0.82125    0.820917         3.942
SVC(C=100, gamma=0.0001)                   rbf       100    0.0001          0.855            0.818438   0.819266         7.95704
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.906875         0.817187   0.816731         3.08272
SVC(C=1, gamma=0.001, kernel='linear')     linear      1    0.001           0.906875         0.817187   0.816731         3.73897
SVC(C=1, gamma=0.0001, kernel='linear')    linear      1    0.0001          0.906875         0.817187   0.816731         3.13279
SVC(C=1, gamma=0.1, kernel='linear')       linear      1    0.1             0.906875         0.817187   0.816731         3.82483
SVC(C=10, gamma='auto', kernel='linear')   linear     10    auto            0.921042         0.812813   0.81273          5.843
SVC(C=10, gamma=0.001, kernel='linear')    linear     10    0.001           0.921042         0.812813   0.81273          4.74299
SVC(C=10, gamma=0.0001, kernel='linear')   linear     10    0.0001          0.921042         0.812813   0.81273          4.262
SVC(C=10, gamma=0.1, kernel='linear')      linear     10    0.1             0.921042         0.812813   0.81273          5.27188
SVC(C=100, gamma='auto', kernel='linear')  linear    100    auto            0.927708         0.805937   0.806154        20.766
SVC(C=100, gamma=0.001, kernel='linear')   linear    100    0.001           0.927708         0.805937   0.806154        19.9108
SVC(C=100, gamma=0.0001, kernel='linear')  linear    100    0.0001          0.927708         0.805937   0.806154        20.3163
SVC(C=100, gamma=0.1, kernel='linear')     linear    100    0.1             0.927708         0.805937   0.806154        22.613
SVC(C=1, gamma=0.001)                      rbf         1    0.001           0.76375          0.755      0.76074         10.6623
SVC(C=0.1, gamma='auto')                   rbf         0.1  auto            0.827083         0.738125   0.74683         12.655
SVC(C=10, gamma=0.0001)                    rbf        10    0.0001          0.754375         0.739062   0.744879        10.5508
SVC(C=1, gamma=0.001, kernel='poly')       poly        1    0.001           0.724375         0.704375   0.711509         5.73216
SVC(C=0.1, gamma=0.1)                      rbf         0.1  0.1             0.64625          0.529375   0.608442        12.719
SVC(C=1, gamma=0.0001)                     rbf         1    0.0001          0.453125         0.430625   0.386974        12.712
SVC(C=0.1, gamma=0.001)                    rbf         0.1  0.001           0.4225           0.4025     0.345189        12.522
SVC(C=100, gamma=0.0001, kernel='poly')    poly      100    0.0001          0.316667         0.309063   0.264459         6.517
SVC(C=0.1, gamma=0.001, kernel='poly')     poly        0.1  0.001           0.31625          0.30875    0.264118         6.17167
SVC(C=0.1, gamma=0.0001)                   rbf         0.1  0.0001          0.0427083        0.0359375  0.00266852      12.838
SVC(C=0.1, gamma=0.0001, kernel='poly')    poly        0.1  0.0001          0.0427083        0.0359375  0.00266852       6.71629
SVC(C=1, gamma=0.0001, kernel='poly')      poly        1    0.0001          0.0427083        0.0359375  0.00266852       7.087
SVC(C=10, gamma=0.0001, kernel='poly')     poly       10    0.0001          0.0427083        0.0359375  0.00266852       6.309
"""

'       \n20000 data\n                                         Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken\n-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------\nSVC(C=10, gamma=\'auto\')                    rbf        10    auto            1                 0.971     0.970534          49.322\nSVC(C=100, gamma=\'auto\')                   rbf       100    auto            1                 0.971     0.970534          48.168\nSVC(C=1, gamma=\'auto\')                     rbf         1    auto            0.994083          0.965625  0.965049          50.95\nSVC(C=10, gamma=0.1)                       rbf        10    0.1             1                 0.964375  0.963821          59.866\nSVC(C=100, gamma=0.1)                      rbf       100    0.1             1                 0.964375  0.963821          60.613\nSVC(C=1, gamma=0.1)                        rbf         1    0.1  

In [22]:
"""       
BEST 3

20000 data
                                         Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf        10    auto            1                 0.971     0.970534          49.322
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999417          0.942875  0.942055          12.922
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.879333          0.853625  0.851967          15.991

13000 data of 20000
                                           Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf        10    auto            1                0.93625    0.936904         9.58105
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999792         0.896875   0.897533         2.47099
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.889583         0.82125    0.820917         3.64601


8000 data of 20000
                                           Kernel      C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  ---  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf        10  auto            1                0.93625    0.936904        10.6901
SVC(C=1, gamma='auto', kernel='poly')      poly        1  auto            1                0.895312   0.896016         2.54
SVC(C=1, gamma='auto', kernel='linear')    linear      1  auto            0.906875         0.817187   0.816731         3.27099


2000 of 20000
                                           Kernel      C   gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=10, gamma='auto')                    rbf         10    auto            1                0.93625    0.936904        10.4032
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999792         0.896875   0.897533         2.70734
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.889583         0.82125    0.820917         3.53565
"""

"       \nBEST 3\n\n20000 data\n                                         Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken\n-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------\nSVC(C=10, gamma='auto')                    rbf        10    auto            1                 0.971     0.970534          49.322\nSVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.999417          0.942875  0.942055          12.922\nSVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.879333          0.853625  0.851967          15.991\n\n13000 data of 20000\n                                           Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken\n-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------\nSVC(C=10, gamma='auto')              

**Απόδοση Ανάλογα με τον Πυρήνα (Kernel):** 
Παρατηρείται συνέπεια στα αποτελέσματα όσον αφορά τον καλύτερο πυρήνα για κάθε δείγμα δεδομένων. Ο πυρήνας RBF παρουσιάζει συνεπώς την υψηλότερη απόδοση ανεξαρτήτως του μεγέθους των δεδομένων, πράγμα που δείχνει την καταλληλότητά του για το συγκεκριμένο πρόβλημα.

**Επίδραση της Παραμέτρου C και Gamma:** 
Η υψηλότερη απόδοση επιτυγχάνεται σε γενικές γραμμές με υψηλότερες τιμές της παραμέτρου C (π.χ., C=10). Αυτό υποδηλώνει ότι ένα πιο περίπλοκο μοντέλο με λιγότερη τιμωρία για τα λάθη κατά την εκπαίδευση είναι προτιμότερο για αυτό το σετ δεδομένων.

**Σχέση Ακρίβειας και Χρόνου Εκπαίδευσης:**  Ενώ ο πυρήνας RBF έχει την καλύτερη απόδοση, παρατηρείται επίσης ότι ο χρόνος εκπαίδευσης είναι σχετικά υψηλός σε σύγκριση με άλλους πυρήνες όπως ο Poly και ο Linear. Αυτό ενδέχεται να σημαίνει ότι οι πυρήνες RBF απαιτούν περισσότερους υπολογιστικούς πόρους, κάτι που πρέπει να ληφθεί υπόψη σε περιβάλλοντα με περιορισμένους υπολογιστικούς πόρους.

Αυτό μπορεί να υποδηλώνει ότι σε πραγματικές συνθήκες, όπου ο χρόνος εκπαίδευσης και η αποδοτικότητα είναι κρίσιμης σημασίας, ο πυρήνας Poly μπορεί να προτιμηθεί έναντι του RBF, ειδικά αν η μικρή διαφορά στην απόδοση μεταξύ των δύο μοντέλων δεν είναι πολύ σημαντική για την εφαρμογή.

In [ ]:
""" 
Using PCA

20000 data

Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
--------  -----  -------  ----------------  ---------------  ----------  ------------
rbf       100    0.1             1                 0.9325    0.932131        33.3404
rbf        10    0.1             0.999             0.932375  0.932015        34.195
rbf        10    auto            0.999417          0.93225   0.931888        32.01
rbf       100    auto            1                 0.932125  0.931743        35.9103
rbf         1    auto            0.979917          0.9285    0.928284        32.344
rbf         1    0.1             0.976667          0.927375  0.927086        32.225
poly        0.1  0.1             0.952333          0.88875   0.888755         8.93297
poly        0.1  auto            0.95775           0.887625  0.8876           8.42901
poly        1    0.1             0.986667          0.887625  0.88735          9.45307
poly        1    auto            0.990417          0.886625  0.886236         9.487
poly       10    auto            0.998583          0.87625   0.87588         10.4224
poly       10    0.1             0.998             0.876     0.875557         9.50001
poly      100    auto            1                 0.875625  0.875267        10.061
poly      100    0.1             0.999833          0.8755    0.875123        10.676
rbf       100    0.001           0.862667          0.843375  0.843694        26.8851
rbf         0.1  auto            0.876083          0.826125  0.828059        45.378
rbf         0.1  0.1             0.8715            0.8265    0.828031        43.129
rbf        10    0.001           0.7905            0.7785    0.778228        32.5739
rbf       100    0.0001          0.75              0.7425    0.741224        35.4202
linear      0.1  auto            0.7475            0.7335    0.732209        14.439
linear      0.1  0.001           0.7475            0.7335    0.732209        14.479
linear      0.1  0.0001          0.7475            0.7335    0.732209        14.352
linear      0.1  0.1             0.7475            0.7335    0.732209        14.257
linear      1    auto            0.75025           0.733125  0.731707        15.586
linear      1    0.001           0.75025           0.733125  0.731707        15.886
linear      1    0.0001          0.75025           0.733125  0.731707        15.981
linear      1    0.1             0.75025           0.733125  0.731707        15.726
linear     10    auto            0.750833          0.732625  0.731155        32.413
linear     10    0.001           0.750833          0.732625  0.731155        32.308
linear     10    0.0001          0.750833          0.732625  0.731155        29.477
linear     10    0.1             0.750833          0.732625  0.731155        29.21
linear    100    auto            0.750917          0.73225   0.730926       176.198
linear    100    0.001           0.750917          0.73225   0.730926       180.03
linear    100    0.0001          0.750917          0.73225   0.730926       178.791
linear    100    0.1             0.750917          0.73225   0.730926       162.968
rbf         1    0.001           0.716833          0.71525   0.713194        38.174
rbf        10    0.0001          0.7055            0.70275   0.700406        44.6027
poly      100    0.001           0.605917          0.59675   0.617099        19.207
rbf         1    0.0001          0.54975           0.552125  0.531259        47.31
rbf         0.1  0.001           0.54525           0.546125  0.525223        51.3608
poly       10    0.001           0.233917          0.22875   0.251398        25.998
poly        1    0.001           0.06675           0.062375  0.0241883       26.8957
rbf         0.1  0.0001          0.0744167         0.07225   0.014839        51.742
poly        0.1  0.001           0.0574167         0.05275   0.00840709      25.8885
poly        0.1  0.0001          0.0574167         0.05275   0.00840709      25.334
poly        1    0.0001          0.0574167         0.05275   0.00840709      25.674
poly       10    0.0001          0.0574167         0.05275   0.00840709      26.33
poly      100    0.0001          0.0574167         0.05275   0.00840709      25.641


13000 of 20000
SVC                                        Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=100, gamma='auto')                   rbf       100    auto            1                0.910962   0.910829        17.7529
SVC(C=10, gamma='auto')                    rbf        10    auto            0.999615         0.910769   0.910646        16.745
SVC(C=100, gamma=0.1)                      rbf       100    0.1             1                0.910577   0.910406        16.062
SVC(C=10, gamma=0.1)                       rbf        10    0.1             0.999615         0.910385   0.910189        16.026
SVC(C=1, gamma='auto')                     rbf         1    auto            0.979744         0.906731   0.906853        17.017
SVC(C=1, gamma=0.1)                        rbf         1    0.1             0.976923         0.904423   0.904575        17.0047
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             0.956923         0.876154   0.876837         4.46801
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.963462         0.874231   0.874861         4.41503
SVC(C=1, gamma=0.1, kernel='poly')         poly        1    0.1             0.991923         0.866538   0.865957         4.938
SVC(C=1, gamma='auto', kernel='poly')      poly        1    auto            0.993718         0.861923   0.8614           4.483
SVC(C=100, gamma='auto', kernel='poly')    poly      100    auto            1                0.852308   0.851429         4.643
SVC(C=100, gamma=0.1, kernel='poly')       poly      100    0.1             1                0.852308   0.851429         4.624
SVC(C=10, gamma=0.1, kernel='poly')        poly       10    0.1             0.998846         0.851731   0.850864         4.603
SVC(C=10, gamma='auto', kernel='poly')     poly       10    auto            0.999615         0.851538   0.85068          5.405
SVC(C=100, gamma=0.001)                    rbf       100    0.001           0.856282         0.827885   0.828119        12.2414
SVC(C=0.1, gamma=0.1)                      rbf         0.1  0.1             0.847436         0.789615   0.793942        21.941
SVC(C=0.1, gamma='auto')                   rbf         0.1  auto            0.849615         0.784038   0.790151        20.97
SVC(C=10, gamma=0.001)                     rbf        10    0.001           0.775385         0.767115   0.766874        14.579
SVC(C=100, gamma=0.0001)                   rbf       100    0.0001          0.740641         0.736923   0.735642        15.13
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.737051         0.731731   0.730541         6.032
SVC(C=0.1, gamma=0.001, kernel='linear')   linear      0.1  0.001           0.737051         0.731731   0.730541         5.898
SVC(C=0.1, gamma=0.0001, kernel='linear')  linear      0.1  0.0001          0.737051         0.731731   0.730541         5.90901
SVC(C=0.1, gamma=0.1, kernel='linear')     linear      0.1  0.1             0.737051         0.731731   0.730541         5.82199
SVC(C=10, gamma='auto', kernel='linear')   linear     10    auto            0.750385         0.731538   0.730363        14.175
SVC(C=10, gamma=0.001, kernel='linear')    linear     10    0.001           0.750385         0.731538   0.730363        13.465
SVC(C=10, gamma=0.0001, kernel='linear')   linear     10    0.0001          0.750385         0.731538   0.730363        14.571
SVC(C=10, gamma=0.1, kernel='linear')      linear     10    0.1             0.750385         0.731538   0.730363        14.149
SVC(C=100, gamma='auto', kernel='linear')  linear    100    auto            0.752051         0.731154   0.730024        84.216
SVC(C=100, gamma=0.001, kernel='linear')   linear    100    0.001           0.752051         0.731154   0.730024        80.263
SVC(C=100, gamma=0.0001, kernel='linear')  linear    100    0.0001          0.752051         0.731154   0.730024        83.476
SVC(C=100, gamma=0.1, kernel='linear')     linear    100    0.1             0.752051         0.731154   0.730024        82.4676
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.746282         0.730577   0.729484         6.451
SVC(C=1, gamma=0.001, kernel='linear')     linear      1    0.001           0.746282         0.730577   0.729484         6.511
SVC(C=1, gamma=0.0001, kernel='linear')    linear      1    0.0001          0.746282         0.730577   0.729484         6.34001
SVC(C=1, gamma=0.1, kernel='linear')       linear      1    0.1             0.746282         0.730577   0.729484         6.394
SVC(C=1, gamma=0.001)                      rbf         1    0.001           0.696154         0.701346   0.6978          18.2372
SVC(C=10, gamma=0.0001)                    rbf        10    0.0001          0.681923         0.686346   0.683243        17.362
SVC(C=100, gamma=0.001, kernel='poly')     poly      100    0.001           0.512051         0.5125     0.550583         9.30999
SVC(C=1, gamma=0.0001)                     rbf         1    0.0001          0.479103         0.486154   0.462822        22.0975
SVC(C=0.1, gamma=0.001)                    rbf         0.1  0.001           0.468205         0.473846   0.447309        21.969
SVC(C=10, gamma=0.001, kernel='poly')      poly       10    0.001           0.2              0.198269   0.205685        11.794
SVC(C=0.1, gamma=0.0001)                   rbf         0.1  0.0001          0.0647436        0.0553846  0.0181176       24.157
SVC(C=1, gamma=0.001, kernel='poly')       poly        1    0.001           0.0502564        0.0440385  0.0134369       11.602
SVC(C=0.1, gamma=0.001, kernel='poly')     poly        0.1  0.001           0.0425641        0.0382692  0.00283528      11.007
SVC(C=0.1, gamma=0.0001, kernel='poly')    poly        0.1  0.0001          0.0425641        0.0382692  0.00283528      12.278
SVC(C=1, gamma=0.0001, kernel='poly')      poly        1    0.0001          0.0425641        0.0382692  0.00283528      12.941
SVC(C=10, gamma=0.0001, kernel='poly')     poly       10    0.0001          0.0425641        0.0382692  0.00283528      11.506
SVC(C=100, gamma=0.0001, kernel='poly')    poly      100    0.0001          0.0425641        0.0382692  0.00283528      10.803


8000 of 20000

SVC                                        Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             0.974167           0.775    0.77749         0.198008
SVC(C=10, gamma=0.1)                       rbf        10    0.1             1                  0.775    0.777131        0.595002
SVC(C=100, gamma=0.1)                      rbf       100    0.1             1                  0.775    0.777131        0.586998
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.984167           0.76875  0.772778        0.197998
SVC(C=10, gamma='auto')                    rbf        10    auto            1                  0.765    0.766308        0.683991
SVC(C=100, gamma='auto')                   rbf       100    auto            1                  0.765    0.766308        0.609998
SVC(C=1, gamma='auto')                     rbf         1    auto            0.990833           0.76     0.762364        0.629001
SVC(C=1, gamma=0.1)                        rbf         1    0.1             0.988333           0.75625  0.757463        0.616998
SVC(C=100, gamma=0.001)                    rbf       100    0.001           0.8575             0.7525   0.751187        0.474003
SVC(C=1, gamma='auto', kernel='poly')      poly        1    auto            0.998333           0.74125  0.743218        0.211992
SVC(C=1, gamma=0.1, kernel='poly')         poly        1    0.1             0.998333           0.74125  0.742864        0.226001
SVC(C=100, gamma='auto', kernel='poly')    poly      100    auto            1                  0.7275   0.73038         0.210001
SVC(C=100, gamma=0.1, kernel='poly')       poly      100    0.1             1                  0.7275   0.73038         0.208003
SVC(C=10, gamma='auto', kernel='poly')     poly       10    auto            1                  0.7275   0.730304        0.224999
SVC(C=10, gamma=0.1, kernel='poly')        poly       10    0.1             1                  0.7275   0.730286        0.215999
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.823333           0.70625  0.703397        0.239998
SVC(C=1, gamma=0.001, kernel='linear')     linear      1    0.001           0.823333           0.70625  0.703397        0.245991
SVC(C=1, gamma=0.0001, kernel='linear')    linear      1    0.0001          0.823333           0.70625  0.703397        0.244994
SVC(C=1, gamma=0.1, kernel='linear')       linear      1    0.1             0.823333           0.70625  0.703397        0.241002
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.761667           0.7025   0.696976        0.21
SVC(C=0.1, gamma=0.001, kernel='linear')   linear      0.1  0.001           0.761667           0.7025   0.696976        0.217
SVC(C=0.1, gamma=0.0001, kernel='linear')  linear      0.1  0.0001          0.761667           0.7025   0.696976        0.214995
SVC(C=0.1, gamma=0.1, kernel='linear')     linear      0.1  0.1             0.761667           0.7025   0.696976        0.195006
SVC(C=10, gamma='auto', kernel='linear')   linear     10    auto            0.85               0.685    0.681207        0.507
SVC(C=10, gamma=0.001, kernel='linear')    linear     10    0.001           0.85               0.685    0.681207        0.510001
SVC(C=10, gamma=0.0001, kernel='linear')   linear     10    0.0001          0.85               0.685    0.681207        0.479999
SVC(C=10, gamma=0.1, kernel='linear')      linear     10    0.1             0.85               0.685    0.681207        0.488001
SVC(C=10, gamma=0.001)                     rbf        10    0.001           0.745              0.68375  0.67543         0.617001
SVC(C=100, gamma='auto', kernel='linear')  linear    100    auto            0.859167           0.67625  0.673333        3.535
SVC(C=100, gamma=0.001, kernel='linear')   linear    100    0.001           0.859167           0.67625  0.673333        3.305
SVC(C=100, gamma=0.0001, kernel='linear')  linear    100    0.0001          0.859167           0.67625  0.673333        3.35001
SVC(C=100, gamma=0.1, kernel='linear')     linear    100    0.1             0.859167           0.67625  0.673333        3.58899
SVC(C=100, gamma=0.0001)                   rbf       100    0.0001          0.7225             0.66875  0.660284        0.574997
SVC(C=1, gamma=0.001)                      rbf         1    0.001           0.520833           0.48375  0.460986        0.559999
SVC(C=10, gamma=0.0001)                    rbf        10    0.0001          0.5225             0.48     0.457438        0.606999
SVC(C=100, gamma=0.001, kernel='poly')     poly      100    0.001           0.280833           0.24625  0.280019        0.266011
SVC(C=0.1, gamma=0.1)                      rbf         0.1  0.1             0.138333           0.0825   0.0448622       0.601997
SVC(C=0.1, gamma='auto')                   rbf         0.1  auto            0.1175             0.0625   0.0429308       0.702
SVC(C=1, gamma=0.0001)                     rbf         1    0.0001          0.115              0.07375  0.0272624       0.607012
SVC(C=10, gamma=0.001, kernel='poly')      poly       10    0.001           0.0525             0.03875  0.0188029       0.279
SVC(C=0.1, gamma=0.001)                    rbf         0.1  0.001           0.0841667          0.055    0.0124879       0.614002
SVC(C=0.1, gamma=0.0001)                   rbf         0.1  0.0001          0.0483333          0.02875  0.00214973      0.625002
SVC(C=0.1, gamma=0.001, kernel='poly')     poly        0.1  0.001           0.0483333          0.02875  0.00214973      0.267008
SVC(C=0.1, gamma=0.0001, kernel='poly')    poly        0.1  0.0001          0.0483333          0.02875  0.00214973      0.248991
SVC(C=1, gamma=0.001, kernel='poly')       poly        1    0.001           0.0483333          0.02875  0.00214973      0.297004
SVC(C=1, gamma=0.0001, kernel='poly')      poly        1    0.0001          0.0483333          0.02875  0.00214973      0.260993
SVC(C=10, gamma=0.0001, kernel='poly')     poly       10    0.0001          0.0483333          0.02875  0.00214973      0.263001
SVC(C=100, gamma=0.0001, kernel='poly')    poly      100    0.0001          0.0483333          0.02875  0.00214973      0.250988


2000 of 20000
SVC                                        Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             0.974167           0.775    0.77749         0.192997
SVC(C=10, gamma=0.1)                       rbf        10    0.1             1                  0.775    0.777131        0.574992
SVC(C=100, gamma=0.1)                      rbf       100    0.1             1                  0.775    0.777131        0.672005
SVC(C=0.1, gamma='auto', kernel='poly')    poly        0.1  auto            0.984167           0.76875  0.772778        0.242
SVC(C=10, gamma='auto')                    rbf        10    auto            1                  0.765    0.766308        0.550004
SVC(C=100, gamma='auto')                   rbf       100    auto            1                  0.765    0.766308        0.564037
SVC(C=1, gamma='auto')                     rbf         1    auto            0.990833           0.76     0.762364        0.634998
SVC(C=1, gamma=0.1)                        rbf         1    0.1             0.988333           0.75625  0.757463        0.610002
SVC(C=100, gamma=0.001)                    rbf       100    0.001           0.8575             0.7525   0.751187        0.416966
SVC(C=1, gamma='auto', kernel='poly')      poly        1    auto            0.998333           0.74125  0.743218        0.192005
SVC(C=1, gamma=0.1, kernel='poly')         poly        1    0.1             0.998333           0.74125  0.742864        0.206002
SVC(C=100, gamma='auto', kernel='poly')    poly      100    auto            1                  0.7275   0.73038         0.188011
SVC(C=100, gamma=0.1, kernel='poly')       poly      100    0.1             1                  0.7275   0.73038         0.20801
SVC(C=10, gamma='auto', kernel='poly')     poly       10    auto            1                  0.7275   0.730304        0.196013
SVC(C=10, gamma=0.1, kernel='poly')        poly       10    0.1             1                  0.7275   0.730286        0.191002
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.823333           0.70625  0.703397        0.249003
SVC(C=1, gamma=0.001, kernel='linear')     linear      1    0.001           0.823333           0.70625  0.703397        0.234996
SVC(C=1, gamma=0.0001, kernel='linear')    linear      1    0.0001          0.823333           0.70625  0.703397        0.249998
SVC(C=1, gamma=0.1, kernel='linear')       linear      1    0.1             0.823333           0.70625  0.703397        0.223002
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.761667           0.7025   0.696976        0.211985
SVC(C=0.1, gamma=0.001, kernel='linear')   linear      0.1  0.001           0.761667           0.7025   0.696976        0.239999
SVC(C=0.1, gamma=0.0001, kernel='linear')  linear      0.1  0.0001          0.761667           0.7025   0.696976        0.216002
SVC(C=0.1, gamma=0.1, kernel='linear')     linear      0.1  0.1             0.761667           0.7025   0.696976        0.194001
SVC(C=10, gamma='auto', kernel='linear')   linear     10    auto            0.85               0.685    0.681207        0.484998
SVC(C=10, gamma=0.001, kernel='linear')    linear     10    0.001           0.85               0.685    0.681207        0.538004
SVC(C=10, gamma=0.0001, kernel='linear')   linear     10    0.0001          0.85               0.685    0.681207        0.582992
SVC(C=10, gamma=0.1, kernel='linear')      linear     10    0.1             0.85               0.685    0.681207        0.517998
SVC(C=10, gamma=0.001)                     rbf        10    0.001           0.745              0.68375  0.67543         0.450994
SVC(C=100, gamma='auto', kernel='linear')  linear    100    auto            0.859167           0.67625  0.673333        3.263
SVC(C=100, gamma=0.001, kernel='linear')   linear    100    0.001           0.859167           0.67625  0.673333        3.356
SVC(C=100, gamma=0.0001, kernel='linear')  linear    100    0.0001          0.859167           0.67625  0.673333        3.117
SVC(C=100, gamma=0.1, kernel='linear')     linear    100    0.1             0.859167           0.67625  0.673333        3.036
SVC(C=100, gamma=0.0001)                   rbf       100    0.0001          0.7225             0.66875  0.660284        0.489
SVC(C=1, gamma=0.001)                      rbf         1    0.001           0.520833           0.48375  0.460986        0.560002
SVC(C=10, gamma=0.0001)                    rbf        10    0.0001          0.5225             0.48     0.457438        0.540028
SVC(C=100, gamma=0.001, kernel='poly')     poly      100    0.001           0.280833           0.24625  0.280019        0.261992
SVC(C=0.1, gamma=0.1)                      rbf         0.1  0.1             0.138333           0.0825   0.0448622       0.619999
SVC(C=0.1, gamma='auto')                   rbf         0.1  auto            0.1175             0.0625   0.0429308       0.561999
SVC(C=1, gamma=0.0001)                     rbf         1    0.0001          0.115              0.07375  0.0272624       0.578998
SVC(C=10, gamma=0.001, kernel='poly')      poly       10    0.001           0.0525             0.03875  0.0188029       0.238995
SVC(C=0.1, gamma=0.001)                    rbf         0.1  0.001           0.0841667          0.055    0.0124879       0.636002
SVC(C=0.1, gamma=0.0001)                   rbf         0.1  0.0001          0.0483333          0.02875  0.00214973      0.742001
SVC(C=0.1, gamma=0.001, kernel='poly')     poly        0.1  0.001           0.0483333          0.02875  0.00214973      0.308999
SVC(C=0.1, gamma=0.0001, kernel='poly')    poly        0.1  0.0001          0.0483333          0.02875  0.00214973      0.268999
SVC(C=1, gamma=0.001, kernel='poly')       poly        1    0.001           0.0483333          0.02875  0.00214973      0.255
SVC(C=1, gamma=0.0001, kernel='poly')      poly        1    0.0001          0.0483333          0.02875  0.00214973      0.245995
SVC(C=10, gamma=0.0001, kernel='poly')     poly       10    0.0001          0.0483333          0.02875  0.00214973      0.23997
SVC(C=100, gamma=0.0001, kernel='poly')    poly      100    0.0001          0.0483333          0.02875  0.00214973      0.233984




"""

In [ ]:
"""     
Using PCA

20000 data

Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
--------  -----  -------  ----------------  ---------------  ----------  ------------
rbf       100    0.1             1                 0.9325    0.932131        33.3404
poly        0.1  0.1             0.952333          0.88875   0.888755         8.93297
linear      0.1  auto            0.7475            0.7335    0.732209        14.439



13000 of 20000
SVC                                        Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=100, gamma='auto')                   rbf       100    auto            1                0.910962   0.910829        17.7529
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             0.956923         0.876154   0.876837         4.46801
SVC(C=0.1, gamma='auto', kernel='linear')  linear      0.1  auto            0.737051         0.731731   0.730541         6.032


8000 of 20000

SVC                                        Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             0.974167           0.775    0.77749         0.198008
SVC(C=10, gamma=0.1)                       rbf        10    0.1             1                  0.775    0.777131        0.595002
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.823333           0.70625  0.703397        0.239998


2000 of 20000
SVC                                        Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
-----------------------------------------  --------  -----  -------  ----------------  ---------------  ----------  ------------
SVC(C=0.1, gamma=0.1, kernel='poly')       poly        0.1  0.1             0.974167           0.775    0.77749         0.192997
SVC(C=10, gamma=0.1)                       rbf        10    0.1             1                  0.775    0.777131        0.574992
SVC(C=1, gamma='auto', kernel='linear')    linear      1    auto            0.823333           0.70625  0.703397        0.249003




"""

**Επιδόσεις χωρίς PCA:**

Ο πυρήνας RBF με C=10 και gamma 'auto' παρουσιάζει την καλύτερη επίδοση σε όλα τα batches δεδομένων (20000, 13000, 8000, και 2000), συγκεκριμένα με F1 Score περίπου 0.936-0.971. Αυτό υποδηλώνει ότι ο πυρήνας RBF είναι ιδιαίτερα αποδοτικός σε προβλήματα όπως η αναγνώριση γραμμάτων.

Ο πυρήνας Poly με C=0.1 και gamma 'auto' παρουσιάζει επίσης καλή επίδοση, ειδικά στα μεγαλύτερα batches δεδομένων (20000 και 13000), με F1 Score περίπου 0.897-0.943.

Ο πυρήνας Linear με C=1 και gamma 'auto' δείχνει χαμηλότερη επίδοση σε σύγκριση με τους άλλους δύο πυρήνες, με F1 Score περίπου 0.821-0.854.

**Επιδόσεις με PCA(0.9):**

Η χρήση PCA φαίνεται να μειώνει την επίδοση των μοντέλων συγκριτικά με τα αποτελέσματα χωρίς PCA. Αυτό μπορεί να οφείλεται στη μείωση της πληροφορίας που παρέχουν τα δεδομένα λόγω της μείωσης διαστάσεων.
Ο πυρήνας RBF με C=100 και gamma 'auto' δείχνει την καλύτερη επίδοση στα μεγαλύτερα batches δεδομένων (20000 και 13000) μετά την εφαρμογή PCA, με F1 Score περίπου 0.911-0.932.

Στα μικρότερα batches δεδομένων (8000 και 2000), ο πυρήνας Poly με C=0.1 και gamma 0.1 φαίνεται να παρουσιάζει καλύτερη επίδοση από τον πυρήνα RBF, με F1 Score περίπου 0.777.

**Γενικά Συμπεράσματα:**

Ο πυρήνας RBF φαίνεται να είναι ιδιαίτερα αποδοτικός για την αναγνώριση γραμμάτων στο dataset Letter Recognition, ειδικά όταν δεν εφαρμόζεται PCA.

Ο πυρήνας Poly είναι ανταγωνιστικός και συγκρίσιμος με τον πυρήνα RBF, ειδικά σε μεγαλύτερα batches δεδομένων.
Η χρήση PCA φαίνεται να μειώνει την συνολική απόδοση των μοντέλων, πιθανόν λόγω απώλειας σημαντικών πληροφοριών που είναι απαραίτητες για την ακριβή ταξινόμηση.

Αξιολογηση με βαση τον χρονο

**Χωρίς PCA:**

Τα μοντέλα με πυρήνα Poly και γάμμα 'auto' παρουσιάζουν γενικά χαμηλότερους χρόνους εκπαίδευσης σε σύγκριση με τα μοντέλα με πυρήνα RBF και Linear. Για παράδειγμα, στα δεδομένα των 20000, το μοντέλο με πυρήνα Poly και C=0.1 διαρκεί περίπου 12.9 λεπτά, ενώ το μοντέλο με πυρήνα RBF και C=10 διαρκεί περίπου 49.3 λεπτά.

Αν και τα μοντέλα με πυρήνα RBF εμφανίζουν καλύτερες επιδόσεις σε όρους F1 Score, τα μεγαλύτερα χρόνια εκπαίδευσης τους μπορεί να αποτελέσουν πρόβλημα σε εφαρμογές με χρονικούς περιορισμούς.

**Με PCA(0.9):**

Η εφαρμογή PCA φαίνεται να μειώνει τους χρόνους εκπαίδευσης σε σχέση με τα μοντέλα χωρίς PCA. Για παράδειγμα, τα μοντέλα με πυρήνα Poly και γάμμα 0.1 στα δεδομένα των 8000 και 2000 εμφανίζουν πολύ χαμηλούς χρόνους εκπαίδευσης (κάτω από 1 λεπτό).

Ωστόσο, παρόλο που η εφαρμογή PCA μειώνει τον χρόνο εκπαίδευσης, τα μοντέλα με PCA εμφανίζουν χαμηλότερη απόδοση σε σύγκριση με τα μοντέλα χωρίς PCA.

**Γενικά Συμπεράσματα:**

Υπάρχει μια σαφής εμπορική ανταλλαγή μεταξύ χρόνου εκπαίδευσης και απόδοσης. Τα μοντέλα με ταχύτερους χρόνους εκπαίδευσης τείνουν να έχουν χαμηλότερη απόδοση, ενώ τα μοντέλα με καλύτερη απόδοση απαιτούν περισσότερο χρόνο.

Ενώ η εφαρμογή PCA μπορεί να είναι ωφέλιμη για τη μείωση των χρόνων εκπαίδευσης, η απώλεια σημαντικής πληροφορίας μπορεί να οδηγήσει σε χειρότερη απόδοση των μοντέλων.